IMPORTING THE REQUITED LIBRARIES

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

DEFINING THE PROPERTIES

In [ ]:
# Defing Properties
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
INITIAL_EPOCHS = 15
FINE_TUNE_EPOCHS = 10
NUM_CLASSES = 7  # Types of cancer
INITIAL_LEARNING_RATE = 1e-3
FINE_TUNE_LEARNING_RATE = 1e-4

MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


DEFINING THE PATHS

In [ ]:
MAIN_FOLDER = Path("/content/drive/MyDrive/Skin-cancer/data")
TRAIN_FOLDER = MAIN_FOLDER / "train"
TEST_FOLDER = MAIN_FOLDER / "test"
VALIDATION_FOLDER = MAIN_FOLDER / "validation"


CREATING THE MODEL

In [ ]:
MAIN_FOLDER = Path("/content/drive/MyDrive/Skin-cancer/data")
TRAIN_FOLDER = MAIN_FOLDER / "train"
TEST_FOLDER = MAIN_FOLDER / "test"
VALIDATION_FOLDER = MAIN_FOLDER / "validation"

CREATING DATA GENERATORS

In [ ]:
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    test_val_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        TRAIN_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    validation_generator = test_val_datagen.flow_from_directory(
        VALIDATION_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    test_generator = test_val_datagen.flow_from_directory(
        TEST_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    return train_generator, validation_generator, test_generator

PLOTTING TRAINING HISTORY

In [ ]:
def plot_training_history(history, fine_tune_history=None):
     # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    if fine_tune_history:
        plt.plot(range(len(history.history['accuracy']), len(history.history['accuracy']) + len(fine_tune_history.history['accuracy'])),
                 fine_tune_history.history['accuracy'])
        plt.plot(range(len(history.history['val_accuracy']), len(history.history['val_accuracy']) + len(fine_tune_history.history['val_accuracy'])),
                 fine_tune_history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation', 'Train Fine-tune', 'Validation Fine-tune'], loc='lower right')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    if fine_tune_history:
        plt.plot(range(len(history.history['loss']), len(history.history['loss']) + len(fine_tune_history.history['loss'])),
                 fine_tune_history.history['loss'])
        plt.plot(range(len(history.history['val_loss']), len(history.history['val_loss']) + len(fine_tune_history.history['val_loss'])),
                 fine_tune_history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation', 'Train Fine-tune', 'Validation Fine-tune'], loc='upper right')

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()



TRAINING AND FINETUNING THE MODEL

In [ ]:
def unfreeze_model(model):
    for layer in model.layers[-50:]:
        layer.trainable = True

def train_model():
    model = create_model()
    model.compile(optimizer=Adam(learning_rate=INITIAL_LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    train_generator, validation_generator, test_generator = create_data_generators()

    # Compute class weights
    class_counts = train_generator.classes  # Get class distribution
    class_weights = compute_class_weight('balanced', classes=np.unique(class_counts), y=class_counts)
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    print("Computed class weights:", class_weights_dict)

    lr_reducer = ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7, verbose=1)
    early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=INITIAL_EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        class_weight=class_weights_dict,  # Added class weights
        callbacks=[lr_reducer, early_stopping]
    )

    # Fine-tuning
    unfreeze_model(model)
    model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history_fine = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=FINE_TUNE_EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        class_weight=class_weights_dict,  # Added class weights
        callbacks=[lr_reducer, early_stopping]
    )

    # Evaluation
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test accuracy: {test_accuracy:.4f}")

    model.save('ham10000_mobilenetv3large_model2.keras')

if __name__ == "__main__":
    train_model()


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 8041 images belonging to 7 classes.
Found 999 images belonging to 7 classes.
Found 985 images belonging to 7 classes.
Computed class weights: {0: np.float64(4.318474758324382), 1: np.float64(2.857498223169865), 2: np.float64(1.3173328964613369), 3: np.float64(12.09172932330827), 4: np.float64(1.2935971685971686), 5: np.float64(0.21280368390409146), 6: np.float64(9.572619047619048)}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 2569s 10s/step - accuracy: 0.3991 - loss: 2.0700 - val_accuracy: 0.6462 - val_loss: 0.9872 - learning_rate: 0.0010
Epoch 2/15
  1/251 ━━━━━━━━━━━━━━━━━━━━ 4:40 1s/step - accuracy: 0.5312 - loss: 0.8430

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


251/251 ━━━━━━━━━━━━━━━━━━━━ 44s 173ms/step - accuracy: 0.5312 - loss: 0.8430 - val_accuracy: 0.6442 - val_loss: 0.9990 - learning_rate: 0.0010
Epoch 3/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 502s 2s/step - accuracy: 0.5556 - loss: 1.2566 - val_accuracy: 0.6371 - val_loss: 0.9750 - learning_rate: 0.0010
Epoch 4/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 51s 200ms/step - accuracy: 0.6562 - loss: 0.7753 - val_accuracy: 0.6169 - val_loss: 1.0311 - learning_rate: 0.0010
Epoch 5/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 497s 2s/step - accuracy: 0.5613 - loss: 1.1808 - val_accuracy: 0.6300 - val_loss: 0.9651 - learning_rate: 0.0010
Epoch 6/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 44s 171ms/step - accuracy: 0.5312 - loss: 0.8732 - val_accuracy: 0.6401 - val_loss: 0.9411 - learning_rate: 0.0010
Epoch 7/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 561s 2s/step - accuracy: 0.6043 - loss: 1.0490 - val_accuracy: 0.5907 - val_loss: 1.0515 - learning_rate: 0.0010
Epoch 8/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 43s 169ms/step - accuracy: 0.7500 - loss: 0.655

LOADING THE TRAINED MODEL

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("ham10000_mobilenetv3large_model2.keras")


EVALUATING THE MODEL ACCURACY

In [ ]:
# Evaluate on training set
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



252/252 ━━━━━━━━━━━━━━━━━━━━ 451s 2s/step - accuracy: 0.8140 - loss: 0.5017
Train Accuracy: 81.56%
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.7568 - loss: 0.5986
Validation Accuracy: 74.77%
31/31 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.7808 - loss: 0.6367
Test Accuracy: 78.07%


CHECKING PREDICTION SCORE MANUALLY

In [ ]:
import numpy as np

def count_correct_predictions(generator, dataset_name):
    y_pred = []
    y_true = []

    # Extract predictions and true labels
    for images, labels in generator:
        preds = model.predict(images)
        y_pred.extend(np.argmax(preds, axis=1))  # Predicted class
        y_true.extend(np.argmax(labels, axis=1))  # True class

        if len(y_true) >= generator.samples:
            break  # Stop once all samples are processed

    # Convert to numpy arrays
    y_pred = np.array(y_pred[:generator.samples])  # Ensure exact number
    y_true = np.array(y_true[:generator.samples])

    correct = np.sum(y_pred == y_true)  # Count correct predictions
    total = generator.samples

    print(f"\n🔍 {dataset_name} - Manual Check:")
    print(f"✅ Correct Predictions: {correct}/{total} ({(correct/total)*100:.2f}%)")

count_correct_predictions(train_generator, "Train")
count_correct_predictions(validation_generator, "Validation")
count_correct_predictions(test_generator, "Test")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 996ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 919ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 927ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 938ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 941ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 953ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 902ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 954ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 906ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 937ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 916ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 928ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 920ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 972ms/step
1/1 ━━━